In [2]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [9]:
import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
lowrankdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/Top10/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))
no_snps_subs = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/fc_subs_no_snps.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []
subids = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    if sub in no_snps_subs:
        continue
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        subids.append(sub)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)
age = (age-np.mean(age))/np.std(age)

print([a.shape for a in [rest, nback, emoid, race, sex, age]])

[(1155, 34716), (1155, 34716), (1155, 34716), (1155,), (1155,), (1155,)]


In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.enc1 = nn.Linear(34716, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, 30).float().cuda()
        self.dec1 = nn.Linear(30, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, 34716).float().cuda()

    def enc(self, x):
        z = F.relu(self.enc1(x))
        z = self.enc2(z)
        return z

    def forward(self, x):
        z = self.enc(x)
        z = F.relu(self.dec1(z))
        z = self.dec2(z)
        return z

pncae = AE()
optim = torch.optim.Adam(pncae.parameters(), lr=1e-4, weight_decay=0)

nepochs = 1000
pperiod = 50

x = np.concatenate([rest, nback, emoid])
x = torch.from_numpy(x).float().cuda()

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

for e in range(nepochs):
    optim.zero_grad()
    xhat = pncae(x)
    loss = rmse(x, xhat)
    loss.backward()
    optim.step()
    if e%pperiod == 0 or e == nepochs-1:
        print(f'{e} {pretty(loss)}')

print('Done')

0 0.3705
50 0.1844
100 0.1808
150 0.1773
200 0.1748
250 0.1718
300 0.1686
350 0.166
400 0.164
450 0.1623
500 0.1609
550 0.1598
600 0.159
650 0.1583
700 0.1576
750 0.1568
800 0.1561
850 0.1554
900 0.1548
950 0.1542
999 0.1537
Done


In [16]:
import scipy.stats as stats

x = torch.from_numpy(emoid).float().cuda()
zz = pncae.enc(x).detach().cpu().numpy()

def corr(z, y):
    z = z - np.mean(z, axis=0, keepdims=True)
    y = y - np.mean(y)
    xx = np.einsum('na,na->a', z, z)
    yy = np.einsum('n,n->', y, y)
    xy = np.einsum('na,n->a', z, y)
    rho = xy/((xx*yy)**0.5)
    n = z.shape[0]
    m = z.shape[1]
    df = n-2
    t = rho*(df/(1-rho**2))**0.5
    t[t < 0] = -t[t < 0]
    p = (1-stats.t.cdf(t, df))*2
    p *= m
    p[p > 1] = 1
    return rho, p

def to_cat(y):
    cats = set()
    for yy in y:
        cats.add(yy)
    cats = list(cats)
    yp = np.zeros(len(y))
    for j,yy in enumerate(y):
        for i,cat in enumerate(cats):
            if yy == cat:
                yp[j] = i
    return yp

demo = pncdemo

n = 0
m = 0
for key in demo.keys():
    z = []
    y = []
    for i,sub in enumerate(subids):
        if sub in demo[key]:
            z.append(zz[i])
            y.append(demo[key][sub])
    if len(z) == 0:
        continue
    n += 1
    nn = len(z)
    z = np.stack(z)
    y = np.array(y)
    if isinstance(y[0], str):
        y = to_cat(y)
    rho, p = corr(z, y)
    if np.any(p < 0.01):
        m += 1
        print(key, 'yes', min(p), nn)
print(n, m)

Race yes 0.0 1155
Sex yes 2.840619939270539e-07 1155
age_at_cnb yes 0.0 1155
PADT_GENUS yes 3.508635699978058e-05 1155
PADT_A yes 1.5514256546111937e-11 1145
PADT_SAME_CR yes 2.6609727754589585e-08 1145
PADT_PC yes 6.661338147750939e-15 1145
PADT_SAME_PC yes 1.575244126472697e-05 1145
PFMT_TN yes 1.9493108593948705e-05 1151
PFMT_FP yes 1.9493108593948705e-05 1151
PFMT_TPRT yes 0.0057581807638174 1151
PFMT_IFAC_TOT yes 4.7126869073821354e-08 1151
PEIT_GENUS yes 0.0003082078464045068 1155
PEIT_CRT yes 9.294428160444568e-07 1155
PEITFEARRT yes 0.0032407584609006612 1151
PEITHAPRT yes 5.543475412039811e-07 1155
PEITSADRT yes 0.004939009575428699 1154
PWMT_TN yes 0.0029143728733571628 1152
PWMT_FP yes 0.0029143728733571628 1152
PWMT_TPRT yes 1.6393486568233584e-09 1152
PWMT_TNRT yes 0.0016372100868222716 1150
PWMT_KIWRD_TOT yes 0.0016340546080573226 1152
PWMT_KIWRD_RTC yes 1.983783610715051e-06 1152
PVRT_GENUS yes 0.0022968279676649495 1155
PVRT_VALID yes 0.0009514776168551009 1155
PVRT_CR 

/tmp/ipykernel_285239/1936710369.py:12: RuntimeWarning: invalid value encountered in divide
  rho = xy/((xx*yy)**0.5)


In [39]:
import pickle
import numpy as np

# s1.0 = male probably

bsnipdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/BSNIP/'
demo = pickle.load(open(f'{bsnipdir}/demographics.pkl', 'rb'))

bfc = []
brace = []
bsex = []
bage = []
bsz = []
subids = []

a,b = np.triu_indices(264,1)

for sub in demo['Age_cal']:
    diag = demo['DXGROUP_1'][sub]
    if diag not in ['SZP', 'NC']:
        continue
    ra = demo['Race'][sub]
    if ra not in ['AA', 'CA']:
        continue
    se = demo['sex'][sub]
    diag = diag == 'SZP'
    ra = ra == 'AA'
    se = se == 's1.0'
    ag = demo['Age_cal'][sub]
    p = np.load(f'{bsnipdir}/fc/{sub}_task-unk_fc.npy')
    p = remap(vec2mat(p))[a,b]
    bage.append(ag)
    bsex.append(se)
    brace.append(ra)
    bsz.append(diag)
    bfc.append(p)
    subids.append(sub)

bfc = np.stack(bfc)
brace = np.array(brace).astype('int')
bsex = np.array(bsex).astype('int')
bsz = np.array(bsz).astype('int')
bage = np.array(bage)

print(max(bage))
print(min(bage))
print(np.mean(bage))
print(np.std(bage))
# bage = (bage-np.mean(bage))/np.std(bage)

print([a.shape for a in [bfc, bsz, brace, bsex, bage]])

64.0
15.0
36.92098765432099
12.335582538145621
[(405, 34716), (405,), (405,), (405,), (405,)]


In [19]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.enc1 = nn.Linear(34716, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, 30).float().cuda()
        self.dec1 = nn.Linear(30, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, 34716).float().cuda()

    def enc(self, x):
        z = F.relu(self.enc1(x))
        z = self.enc2(z)
        return z

    def forward(self, x):
        z = self.enc(x)
        z = F.relu(self.dec1(z))
        z = self.dec2(z)
        return z

bsnipae = AE()
optim = torch.optim.Adam(bsnipae.parameters(), lr=1e-4, weight_decay=0)

nepochs = 1000
pperiod = 20

x = torch.from_numpy(bfc).float().cuda()

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

for e in range(nepochs):
    optim.zero_grad()
    xhat = bsnipae(x)
    loss = rmse(x, xhat)
    loss.backward()
    optim.step()
    if e%pperiod == 0 or e == nepochs-1:
        print(f'{e} {pretty(loss)}')

print('Done')

0 0.3166
20 0.2179
40 0.1874
60 0.1842
80 0.1827
100 0.1812
120 0.1794
140 0.177
160 0.1741
180 0.1715
200 0.1691
220 0.1671
240 0.1651
260 0.1632
280 0.1614
300 0.1596
320 0.1578
340 0.1559
360 0.154
380 0.152
400 0.1501
420 0.1481
440 0.1461
460 0.1442
480 0.142
500 0.1399
520 0.1378
540 0.1356
560 0.1333
580 0.1311
600 0.1288
620 0.1265
640 0.1242
660 0.1219
680 0.1197
700 0.1174
720 0.1152
740 0.1132
760 0.111
780 0.1092
800 0.1069
820 0.1049
840 0.103
860 0.1014
880 0.0994
900 0.0977
920 0.0959
940 0.0942
960 0.0927
980 0.091
999 0.09
Done


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import random

class VAE2(nn.Module):
    def __init__(self, fcd, ld, matsz, rank):
        super(VAE2, self).__init__()
        self.fcd = fcd
        self.ld = ld
        self.matsz = matsz
        self.rank = rank
        self.enc1 = nn.Linear(fcd, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+4, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, matsz*rank).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)
    
    def dec(self, z, age, sex, race, sz):
        z = torch.cat([z, age.unsqueeze(1), sex.unsqueeze(1), race.unsqueeze(1), sz.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x = self.dec2(x)
        x = x.reshape(len(z), self.matsz, self.rank)
        x = torch.einsum('abc,adc->abd', x, x)
        return x

    def vectorize(self, x):
        a,b = np.triu_indices(self.matsz, 1)
        return x[:,a,b]

bsnipvae = VAE2(34716, 30, 264, 5)
bsnipvae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/BSNIP/vae_1000_z30_cov4_264_rank5.torch'))
bsnipvae.eval()

print('Done')

Done


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression

zz = bsnipvae.enc(torch.from_numpy(bfc).float().cuda()).detach().cpu().numpy()

for alpha in [0.01, 0.1,1,10,100,1000]:
    errs = []
    nulls = []

    for _ in range(10):
        xtr, xt, ytr, yt = train_test_split(zz, bage, train_size=0.8)
        
        reg = Ridge(alpha=alpha).fit(xtr, ytr)
        # reg = LogisticRegression(C=alpha, max_iter=100).fit(xtr, ytr)
        yhat = reg.predict(xt)
        # err = np.mean(yt == yhat)
        # null = np.mean(yt)
        # if null < 0.5:
        #     null = 1-null
        err = np.mean((yt-yhat)**2)**0.5
        null = np.mean((np.mean(ytr)-yt)**2)**0.5
        errs.append(err)
        nulls.append(null)

    print(alpha, np.mean(errs), np.std(errs), np.mean(nulls), np.std(nulls))

0.01 13.827669744029055 0.9913958898589317 12.357500175738373 0.431076559303278
0.1 15.173403509023894 1.478275180247408 12.27088175927876 0.4061331804602516
1 14.622717778824907 1.3690201021457171 12.562640725797374 0.6868838639007405
10 13.502826763173932 0.8502474889623365 12.209362324668955 0.4978536206620424
100 12.588501866042233 0.6902210882519809 12.363394756036367 0.6689863470479284
1000 12.669323042247862 0.56315195419504 12.646574238965764 0.5638599336084572
